https://en.wikipedia.org/wiki/Mountain_car_problem

In [17]:
import gym

In [18]:
env = gym.make('MountainCar-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [19]:
observation = env.reset()

In [20]:
observation

array([-0.48438065,  0.        ])

In [21]:
# 3 actions: push left (0), no push (1), push right (2)
env.action_space

Discrete(3)

In [6]:
env.observation_space

Box(2,)

In [7]:
print(env.observation_space.high)
print(env.observation_space.low)

[0.6  0.07]
[-1.2  -0.07]


In [13]:
for i in range(1000):
    env.render()
    env.step(env.action_space.sample())
    

In [11]:
env.step(env.action_space.sample())

(array([-0.76775078,  0.00150308]), -1.0, True, {})

In [27]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 50

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            #print("state: {}, action: {}, reward: {}, next_state: {}, done: {}".format(state, action, reward, next_state, done))
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


if __name__ == "__main__":
    env = gym.make('MountainCar-v0')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    done = False
    batch_size = 32

    for e in range(EPISODES):
        
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        
        for time in range(200):
            # turn this off if you don't want to render
            env.render()
            
            action = agent.act(state)
            
            next_state, reward, done, _ = env.step(action)
            if done:
                print("done within {} time".format(time))
            reward = abs(reward) if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
done within 199 time
episode: 0/50, score: 199, e: 0.43
done within 199 time
episode: 1/50, score: 199, e: 0.16
done within 199 time
episode: 2/50, score: 199, e: 0.059
done within 199 time
episode: 3/50, score: 199, e: 0.022
done within 199 time
episode: 4/50, score: 199, e: 0.01
done within 199 time
episode: 5/50, score: 199, e: 0.01
done within 199 time
episode: 6/50, score: 199, e: 0.01
done within 199 time
episode: 7/50, score: 199, e: 0.01
done within 199 time
episode: 8/50, score: 199, e: 0.01
done within 199 time
episode: 9/50, score: 199, e: 0.01
done within 199 time
episode: 10/50, score: 199, e: 0.01
done within 199 time
episode: 11/50, score: 199, e: 0.01
done within 199 time
episode: 12/50, score: 199, e: 0.01
done within 199 time
episode: 13/50, score: 199, e: 0.01
done within 199 time
episode: 14/50, score: 199, e: 0.01
done within 199 time
episode: 15/50, score: 199, e: 0.